# Mapper

> Maps IOM evaluation report against evaluation frameworks 

Maps evaluation reports to IOM's SRF(Strategic Results Framework) and GCM (Global Compact for Migration UN General Assembly resolution) themes using LLM-based scoring. The module analyzes reports against four theme hierarchies (SRF Enablers, Cross-cutting Priorities, GCM Objectives, and SRF Outputs) using prompt caching for efficiency. Returns structured centrality scores with reasoning and confidence levels for each theme.

In [ ]:
#| default_exp mapper

In [ ]:
#| export
from fastcore.all import *
from pydantic import BaseModel
from lisette.core import completion, mk_msg
from iomeval.core import load_prompt, n_tokens
from iomeval.themes import load_enablers, load_ccp, load_gcms, load_srf_outs, load_gcm_lut, fmt_enablers_ccp, fmt_srf_outs, get_srf_outs, load_all_themes
import json

## Response Models

`ThemeScore` and `ThemeScores` are Pydantic models for structured LLM output. The LLM returns scores for each theme with reasoning and confidence levels.


In [ ]:
#| export
class ThemeScore(BaseModel):
    "Single theme's centrality assessment with score, reasoning, and confidence"
    theme_id: str       # Unique identifier for the theme
    theme_title: str    # Human-readable theme name
    centrality_score: float  # 0-1 score indicating how central theme is to report
    reasoning: str      # LLM's explanation for the score
    confidence: str     # high/medium/low confidence in assessment

In [ ]:
#| export
class ThemeScores(BaseModel):
    "Collection of theme centrality scores from a single mapping operation"
    scores: list[ThemeScore]  # All theme scores for a single mapping call

## Parsing

Helper functions to extract and sort results from the LLM response.


In [ ]:
#| export
def parse_json_response(res): 
    "Extract and parse JSON content from LLM completion response"
    return json.loads(res.choices[0].message.content)

In [ ]:
#| export
def sort_by_centrality(res):
    "Sort themes by centrality score, accepts raw response or parsed dict"
    data = res if isinstance(res, dict) else parse_json_response(res)
    return sorted(data['scores'], key=lambda x: x['centrality_score'], reverse=True)

In [ ]:
#| export
def get_top_ids(res, min_score=0.7):
    "Get IDs of themes with centrality score >= min_score, accepts raw response or parsed dict"
    data = res if isinstance(res, dict) else parse_json_response(res)
    return [o['theme_id'] for o in data['scores'] if o['centrality_score'] >= min_score]

## Core Mapping

The core mapping functions prepare the report for caching and call the LLM to score themes against the report content.

In [ ]:
#| export
def mk_system_blocks(report:str  # Full report text to analyze
                    ) -> list:   # Anthropic-style content blocks with cache control
    "Create cached system message blocks from report text"
    return [{"type": "text", "text": f"## Report to Analyze\n\n{report}", "cache_control": {"type": "ephemeral"}}]

In [ ]:
#| export
def map_themes(system_blocks:list,   # Cached system blocks from mk_system_blocks
               themes:str,            # Formatted themes text to score against
               prompt:str,            # Mapping instruction prompt
               model:str='claude-haiku-4-5',  # Model to use for completion
               response_format=ThemeScores    # Pydantic model for structured output
              ):
    "Map report against themes using cached system blocks"
    return completion(model=model, system=system_blocks, messages=[mk_msg(f"{prompt}\n\n## Themes\n\n{themes}")], 
                     response_format=response_format, max_tokens=8192)

In [ ]:
#| eval: false
from iomeval.extract import extract_sections
from mistocr.core import read_pgs

md = read_pgs('files/test/AAP%20Evaluation%20Report_final_')
report = extract_sections(md)

In [ ]:
#| eval: false
n_tokens(report)

8657

In [ ]:
#| eval: false
system_blocks = mk_system_blocks(report)
print(system_blocks[0]['text'][:200])

## Report to Analyze

## EXECUTIVE SUMMARY  ... page 6

This external evaluation assessed whether or not IOM is delivering on its commitments to accountability to affected populations (AAP), ${ }^{1}$


In [ ]:
#| eval: false
enablers = load_enablers()
prompt = load_prompt('srf_enablers')
res = map_themes(system_blocks, fmt_enablers_ccp(enablers), prompt)
sort_by_centrality(res)[:3]


[{'theme_id': 'Cross-cutting 3',
  'theme_title': 'Funding',
  'centrality_score': 0.81,
  'reasoning': "Funding is a MAJOR COMPONENT throughout the evaluation. Executive Summary states 'Resources allocated to central AAP functions remain inadequate' and notes 'AAP Coordination Unit is largely funded with earmarked funds.' Conclusions emphasize inadequate resource allocation repeatedly: 'The opinion that resources allocated to AAP in IOM are inadequate was prevalent in both qualitative and quantitative data.' Recommendations 2, 4, and 5 directly address funding strategy and mechanisms (e.g., 'three-year plan connected to a funding strategy'; 'Establish a fund for supporting select country offices'). The Ethiopia case study exemplifies office-wide funding approach. Multiple findings address unsustainability risks from fragmented, project-specific funding models.",
  'confidence': 'high'},
 {'theme_id': 'Cross-cutting 4',
  'theme_title': 'Data and evidence',
  'centrality_score': 0.75,


## Pipeline

The full pipeline maps a report against all theme hierarchies: SRF Enablers → Cross-cutting Priorities → GCM Objectives → SRF Outputs. The GCM lookup table filters which outputs to score based on top GCM objectives.

In [ ]:
#| export
def load_prompts(path:str='files/prompts'  # Directory containing prompt files
                ) -> AttrDict:              # Dict with srf_enablers, srf_ccps, gcm, srf_outputs prompts
    "Load all mapping prompts"
    return AttrDict({k: load_prompt(k, path) for k in ['srf_enablers', 'srf_ccps', 'gcm', 'srf_outputs']})

In [ ]:
#| export
@delegates(map_themes)
def map_all(report:str,                      # Full report text to analyze
            path:str='files/themes',         # Directory containing theme JSON files
            prompt_path:str='files/prompts', # Directory containing prompt files
            verbose:bool=True,               # Print progress messages
            **kwargs                         # Additional args passed to map_themes (e.g. model)
           ) -> AttrDict:                    # Dict with enablers, ccp, gcm, outputs results
    "Map report against all theme classes: enablers → CCP → GCM → outputs"
    themes, prompts = load_all_themes(path), load_prompts(prompt_path)
    system_blocks = mk_system_blocks(report)
    
    if verbose: print("Mapping SRF Enablers...")
    enablers_res = map_themes(system_blocks, fmt_enablers_ccp(themes.enablers), prompts.srf_enablers, **kwargs)
    if verbose: print("Mapping Cross-cutting Priorities...")
    ccp_res = map_themes(system_blocks, fmt_enablers_ccp(themes.ccp), prompts.srf_ccps, **kwargs)
    if verbose: print("Mapping GCM Objectives...")
    gcm_res = map_themes(system_blocks, themes.gcms, prompts.gcm, **kwargs)
    
    top_gcm_ids = get_top_ids(gcm_res)
    if not top_gcm_ids:
        if verbose: print("No GCM objectives scored ≥0.7, skipping SRF Outputs")
        return AttrDict(enablers=enablers_res, ccp=ccp_res, gcm=gcm_res, outputs=None)
    
    if verbose: print(f"Top GCM: {top_gcm_ids[0]} (from {len(top_gcm_ids)} candidates)")
    output_ids = get_srf_outs(themes.gcm_lut, [top_gcm_ids[0]])
    if verbose: print(f"Mapping {len(output_ids)} filtered SRF Outputs...")
    outputs_res = map_themes(system_blocks, fmt_srf_outs(themes.srf_outs, output_ids), prompts.srf_outputs, **kwargs)
    return AttrDict(enablers=enablers_res, ccp=ccp_res, gcm=gcm_res, outputs=outputs_res)


In [ ]:
#| eval: false
res = map_all(report)

Mapping SRF Enablers...


Mapping Cross-cutting Priorities...


Mapping GCM Objectives...


Top GCM: 7 (from 2 candidates)
Mapping 26 filtered SRF Outputs...


In [ ]:
#| eval: false
sort_by_centrality(res.enablers)[:2]

[{'theme_id': 'Cross-cutting 3',
  'theme_title': 'Funding',
  'centrality_score': 0.82,
  'reasoning': "Funding is a PRIMARY FOCUS. The evaluation identifies inadequate resource allocation as a central finding, appearing prominently in the Executive Summary: 'Resources allocated to central AAP functions remain inadequate.' Funding is addressed extensively across multiple sections: the AAP Coordination Unit lacks core funding and relies on 'earmarked funds'; country-level AAP is 'project-specific' creating fragmentation; there is 'no strategic approach to AAP funding' (Conclusions). Recommendations 2, 4, and 5 all directly address funding strategy and mechanisms. However, broader financing innovation, multi-year programming, and flexible funding—elements of the enabler description—are not explored, limiting it from 0.9.",
  'confidence': 'high'},
 {'theme_id': 'Cross-cutting 2',
  'theme_title': 'Partnership',
  'centrality_score': 0.68,
  'reasoning': "Partnership is a major component

In [ ]:
#| eval: false
sort_by_centrality(res.ccp)[:2]

[{'theme_id': 'Cross-cutting 1',
  'theme_title': 'Integrity, Transparency and Accountability',
  'centrality_score': 0.95,
  'reasoning': "This is a THEMATIC EVALUATION of AAP, which directly operationalizes accountability to affected populations. The entire evaluation assesses IOM's commitment to 'active commitment to use power responsibly by taking account of, giving account to, and being held to account by the people' (Executive Summary). Accountability appears in evaluation objectives, findings on 'collective accountability' (Conclusions), CFM mechanisms (Recommendation 8), and visibility/transparency (Recommendation 6). The evaluation systematically assesses accountability structures, transparency of feedback mechanisms, and institutional commitment to these principles across multiple sections and recommendations.",
  'confidence': 'high'},
 {'theme_id': 'Cross-cutting 3',
  'theme_title': 'Protection-centred',
  'centrality_score': 0.72,
  'reasoning': "Protection is a MAJOR COM

In [ ]:
#| eval: false
get_top_ids(res.gcm)

['7', '23']

In [ ]:
#| eval: false
sort_by_centrality(res.outputs)[:1]

[{'theme_id': '3d31',
  'theme_title': 'Government and civil society stakeholders have improved knowledge, skills and resources to protect the rights of vulnerable migrants, including children and victims of trafficking, and are equipped to apply a survivor-centered, gender-sensitive, human-rights based approach.',
  'centrality_score': 0.75,
  'reasoning': "Directly relevant to AAP's human rights and vulnerable group protection focus. Report emphasizes AAP's grounding in human rights-based approaches (Conclusions, page 34: 'The desk review and consultations held confirmed that the IOM AAP approach considers the cross-cutting issues of human rights, gender equality and disability'). Findings address capacity gaps (Recommendations 4, 7 on training/support). However, evaluation does not assess actual vulnerable migrant protection outcomes or survivor-centered service delivery—focuses on organizational support capacity.",
  'confidence': 'high'}]

Can you provide a title and a short description for this module?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Based on the code, here's a title and description:

**Title:** Theme Mapper

**Description:** Maps evaluation reports to IOM's Strategic Results Framework themes using LLM-based scoring. The module analyzes reports against four theme hierarchies (SRF Enablers, Cross-cutting Priorities, GCM Objectives, and SRF Outputs) using prompt caching for efficiency. Returns structured centrality scores with reasoning and confidence levels for each theme.